# FIFA World Cup 2026 — Simulated with bracketeer

The 2026 FIFA World Cup introduces an expanded 48-team format: 12 groups of 4,
72 group-stage matches, and a 32-team knockout that plays all the way through
to the final and third-place play-off.

This notebook builds the full competition with bracketeer and simulates it
from kick-off to champion — 104 matches in total.

In [ ]:
if (!requireNamespace("bracketeer", quietly = TRUE)) {
  install.packages("devtools")
  devtools::install_github("bbtheo/bracketeer")
}
library(bracketeer)

## The format

| Phase | Details | Matches |
|---|---|---|
| Group stage | 12 groups of 4, single round-robin | 72 |
| Round of 32 | Top 2 per group + best 8 third-placed teams | 16 |
| Round of 16 | | 8 |
| Quarter-finals | | 4 |
| Semi-finals | | 2 |
| Third-place play-off | | 1 |
| Final | | 1 |
| **Total** | | **104** |

24 teams qualify automatically (top 2 from each of the 12 groups). The
remaining 8 knockout spots go to the best third-placed teams ranked across
all groups by points, then goal difference.

In [ ]:
teams <- c(
  "Canada", "Mexico", "United States",
  "Argentina", "Brazil", "Uruguay", "Colombia", "Ecuador",
  "Paraguay", "Chile",
  "England", "France", "Germany", "Spain", "Portugal",
  "Netherlands", "Belgium", "Croatia", "Italy", "Denmark",
  "Switzerland", "Austria", "Serbia", "Poland", "Ukraine",
  "Japan", "Korea Republic", "IR Iran", "Saudi Arabia",
  "Australia", "Qatar", "Uzbekistan", "Jordan",
  "Morocco", "Senegal", "Tunisia", "Algeria", "Egypt",
  "Cote d'Ivoire", "Ghana", "Cameroon", "Nigeria", "South Africa",
  "New Zealand",
  "Costa Rica", "Panama", "Jamaica", "Honduras"
)

cat(length(teams), "teams registered\n")

## Qualification selector

bracketeer's `filter_by()` accepts a custom function that receives the source
stage's standings and returns the participants to advance. Here we implement
the official qualification rule: top 2 from each group advance automatically,
then the 8 best third-placed finishers are ranked across all 12 groups.

In [ ]:
best_32_selector <- filter_by(function(standings = NULL, source_pool = NULL, ...) {
  if (is.null(standings) || !is.data.frame(standings) ||
      !all(c("participant", "group") %in% names(standings))) {
    pool <- as.character(source_pool)
    return(head(pool, min(length(pool), 32L)))
  }

  tbl <- standings
  tbl$participant <- as.character(tbl$participant)
  tbl$group      <- as.character(tbl$group)

  if (!"group_rank" %in% names(tbl)) {
    tbl <- tbl[order(tbl$group, tbl$rank), , drop = FALSE]
    tbl$group_rank <- as.integer(ave(
      tbl$rank, tbl$group,
      FUN = function(x) rank(x, ties.method = "first")
    ))
  }
  if (!"points" %in% names(tbl))     tbl$points     <- 0L
  if (!"score_diff" %in% names(tbl)) tbl$score_diff <- 0L

  auto_qualifiers <- tbl[tbl$group_rank <= 2L, , drop = FALSE]
  thirds          <- tbl[tbl$group_rank == 3L, , drop = FALSE]
  thirds          <- thirds[order(-thirds$points, -thirds$score_diff), , drop = FALSE]
  best_thirds     <- utils::head(thirds, 8L)

  unique(c(as.character(auto_qualifiers$participant),
           as.character(best_thirds$participant)))
})

## Building the tournament

`spec()` defines the structure without committing to a specific field of teams.
`validate()` checks that the routing is feasible before we materialise anything.
`build()` creates the live tournament runtime.

In [ ]:
wc2026 <- spec() |>
  round_robin(
    "groups",
    groups       = 12,
    tiebreakers  = c("points", "score_diff", "sos", "head_to_head", "alphabetical")
  ) |>
  single_elim("knockout", third_place = TRUE, take = best_32_selector)

validate(wc2026, n = length(teams))

trn <- build(wc2026, teams)
stage_status(trn)

## Group stage

72 matches across 12 groups. Every team plays 3 matches. We simulate each
result with a random score and then manually advance — `auto_advance = FALSE`
here so we can inspect the group standings before the knockout draw.

In [ ]:
set.seed(2026)

random_score <- function(max_goals = 4L, allow_draw = TRUE) {
  s <- sample.int(max_goals + 1L, 2L, replace = TRUE) - 1L
  if (!allow_draw && s[1L] == s[2L]) {
    i    <- sample.int(2L, 1L)
    s[i] <- s[i] + 1L
  }
  as.numeric(s)
}

group_m <- matches(trn, "groups", status = "all")
cat("Playing", nrow(group_m), "group-stage matches ...\n")

for (i in seq_len(nrow(group_m))) {
  trn <- result(trn, "groups",
    match = group_m$match_id[[i]],
    score = random_score(allow_draw = TRUE),
    auto_advance = FALSE
  )
}

trn <- advance(trn, "groups")
cat("Group stage complete.\n")
stage_status(trn)

## Group standings

Final standings for all 12 groups. The top 2 from each group qualify
automatically; 8 additional spots go to the best third-placed teams.

In [ ]:
grp <- standings(trn, "groups")

for (g in sort(unique(grp$group))) {
  rows <- grp[grp$group == g, ]
  rows <- rows[order(rows$rank), ]
  cat(sprintf(
    "Group %s  |  %s  %s  %s  %s\n", g,
    rows$participant[1L], rows$participant[2L],
    rows$participant[3L], rows$participant[4L]
  ))
}

## Knockout stage

32 qualifiers. No draws — every match goes to a winner. The bracket runs
through Round of 32, Round of 16, Quarter-finals, Semi-finals, Third-place
play-off, and the Final.

In [ ]:
rounds_played <- 0L

while (!isTRUE(trn$completed)) {
  playable <- matches(trn, "knockout", status = "pending")
  playable <- playable[
    !is.na(playable$participant1) & !is.na(playable$participant2), ,
    drop = FALSE
  ]
  if (nrow(playable) == 0L) break

  for (mid in playable$match_id) {
    trn <- result(trn, "knockout",
      match = mid,
      score = random_score(allow_draw = FALSE)
    )
  }
  rounds_played <- rounds_played + 1L
}

cat("Knockout complete after", rounds_played, "rounds.\n")
stage_status(trn)

In [ ]:
cat("\n⚽  FIFA World Cup 2026 Champion:", winner(trn), "\n")

## All 104 results

Every match from the group stage through to the final, with scores and the
winning team.

In [ ]:
all_m   <- matches(trn, status = "all")
display <- c("stage_id", "round", "participant1", "score1", "score2",
             "participant2", "winner")
all_m[, intersect(display, names(all_m))]